Ce fichier tout entier sert à stocker les fonctions [communes] qui pourraient etre appeler par le développeur (Data Scientist) dans le cas où il aurait dans l'idée de tester de nouveau alogithmes , tout en gardant notre trame de raisonnement.

In [1]:
%run classes_definitions.ipynb

### importation de modules
L'imporation des modules va permettre de faire appel à n'importe quel fonction contenu à l'intérieur

In [2]:
import math
import re
import numpy as np
import pandas as pd
import random 
import matplotlib.pyplot as plt
import mpld3
from mpld3 import plugins

stockage des données sous forme de matrice (liste de listes) , pour un meilleur acces
par indices dans le but d'extraire des données pour la suite de nos analyses

In [3]:
user = []
item = []
rating = []
rating_test = []

chargement des données du dataset **movielens** et "parsation" en meme temps (découpage par colonnes)

In [4]:
d = Dataset()
d.load_users("data/u.user", user)
d.load_items("data/u.item", item)
d.load_ratings("data/u.base", rating)
d.load_ratings("data/u.test", rating_test)

In [5]:
n_users = len(user)
n_items = len(item)

la **_utility matrix_** sert a stocker la note de chaque user a chaque item sous forme matricielle
son principe est le suivant si user $i$ note item $j$ : 

$$utilitymatrix[i][j] != 0$$
sinon :

$$utilitymatrix[i][j] = 0$$
(c'est un peu la matrice mère)

nous faisons la meme chose pour la matrice _"test"_ qui contient un certain nombre de notes données par les utilisateurs , il s'agit de la matrice qui nous servira à valider la précision de notre estimateur

In [6]:
utility_matrix = np.zeros((n_users, n_items))
for rate in rating:
    utility_matrix[rate.user_id - 1][rate.item_id - 1] = rate.rating

test = np.zeros((n_users, n_items))
for rate in rating_test:
    test[rate.user_id - 1][rate.item_id - 1] = rate.rating

#### Marquage du genre

Cette instruction a pour but de parcourir tous les items contenus dans le fichier **item** et récupérera le genre du film en fonction la position de l'élément diffèrent de zéro [car la dernière case de chaque item est constituée de 0 et de 1 , 1 si l'item $i_0$ est de ce genre là 0 sinon (c'est sur cette liste que nous allons performer notre clustering dans la méthde d'apprentissage automatique)

In [7]:
movie_genre = []
for movie in item:
    movie_genre.append([movie.unknown, movie.action, movie.adventure, movie.animation,
                        movie.childrens, movie.comedy,movie.crime, movie.documentary,
                        movie.drama, movie.fantasy, movie.film_noir, movie.horror,
                        movie.musical, movie.mystery, movie.romance, movie.sci_fi,
                        movie.thriller, movie.war, movie.western])

### Comparaison des estimteurs
Il s'agit d'une fonction dont le seul argument est la méthode de machine learning employé , 
son but est de comparer la proximité entre la note prédite par l'estimateur et la note stocké dans
le fichier _"test"_ en se servant du _Mean Square Error_

#### Mean Square Error
Le MSE sert à mesurer la précision d' un estimateur en servant de son biais et sa variance ,
le meilleur estimateur est souvent (toujours) celui dont la variance est la plus petite  , c'est le meilleur étalon pour des estimateurs de méthode à approche par minimisation de variance (ex : Kmeans..)

#### Homogeneity
Cette métrique sert à vérifier _"l'homogénéité"_ d'une classe en s'assurant que les éléments qui s'y trouvent n'appartiennent qu' a cette classe , son grand avantage est qu'aucune supposition n'est faite sur la composition des clusters.


In [8]:
def metrics_comparison(method):
    y_true = []
    y_pred = []
    f = open('test.txt', 'w')
    for i in range(0, n_users):
        for j in range(0, n_items):
            if test[i][j] > 0:
                f.write("%d, %d, %.4f\n" % (i+1, j+1, utility_copy[i][method[j]-1]))
                y_true.append(test[i][j])
                y_pred.append(utility_copy[i][method[j]-1])
    f.close()
    print("Mean Squared Error of the estimator is: %f" %metrics.mean_squared_error(y_true, y_pred))
    print("Homogeneity is : %f" %metrics.homogeneity_score(y_true, y_pred)) 

### Suggestion des films les mieux notés

Ici l'idée est simple , une fois que notre utilisateur système sait de qui il est proche et quels genres de films lui sont sugérés en priorité il peut encore ne pas savoir où commencer ;il est donc judicieux de lui indiquer une piste pour savoir où regarder en se basant sur les films les mieux notés par son similare toutes catégories (genres) confondues.

In [9]:
def max_rated_user_index_max(index_max , numb_genre):
    lst_items = [[rate.user_id , rate.item_id , rate.rating] for rate in rating]
    df_rate = pd.DataFrame(lst_items)
    df_rate.columns = ['user_id' , 'item_id' , 'rate']
    #constitution du dataframe des user_id des films_id note des films
    df_item = pd.DataFrame([[it.id , it.title] for it in item])
    df_item.columns = ['item_id' , 'movie_title']
    #constitution du dataframe films_id et le titre du film
    list_max_rated = []
    df_rate_user_index_max = df_rate[df_rate['user_id'] == index_max]
    #sous dataframe des notes de l utilisateur index_max uniquement
    max_rate = max(df_rate_user_index_max['rate'])
    #note maximale accorde par l'utilisateur index_max
    df_rate_user_index_max_item_max = df_rate_user_index_max[df_rate_user_index_max['rate'] == max_rate]
    #sous dataframe des notes films_id avec la note maximale
    for item_id in df_rate_user_index_max_item_max['item_id']:
        df = df_item[df_item['item_id'] == item_id]
        dt = df['movie_title'].tolist()
        list_max_rated.append(dt)
    df_movies = pd.DataFrame(list_max_rated[0 : numb_genre])
    df_movies.columns = ['movies']
    return df_movies

